In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import json, codecs
from collections import Counter

It's a simple verification of iwildcam2022_test_information.json

In [ ]:
def extractor(lis, nam):
    length = len(lis)
    result = ['' for i in range(length)]
    for i in range(length):
        result[i] = lis[i][nam]
    return result

def deletor(lis, inde):    
    if len(inde)>0:
        inde.sort()
        lis_new = lis.copy()
        for i in range(len(inde)):
            del lis_new[inde[i]-i]
        return lis_new
    else:
        return lis
    
def ploter(seq_all, num):
    import matplotlib.pyplot as plt
    import matplotlib.image as imgplt
    
    seq = seq_all[num]['image']
    file_name = extractor(seq, 'file_name')
    
    plt.figure(figsize=(30,30))
    for i in range(len(file_name)):            
        x = imgplt.imread('../train/train/' + file_name[i])
        plt.subplot(3,4,i+1)
        plt.imshow(x)

In [ ]:
with codecs.open("../input/iwildcam2022-fgvc9/metadata/metadata/iwildcam2022_train_annotations.json", 'r',
                 encoding='utf-8', errors='ignore') as f:
    train_meta = json.load(f)
               
file_name = extractor(train_meta['images'], 'id')
seq_name = extractor(train_meta['images'], 'seq_id')

file_name2 = extractor(train_meta['annotations'], 'image_id')
cate_name = extractor(train_meta['annotations'], 'category_id')

Reading data

In [ ]:
seq_name_format = dict(Counter(seq_name))
seq_all = ['' for i in range(len(seq_name_format))]

i = 0
seq_loc = list()
seq_loc_empty = list()
seq_loc.append(-1)
seq_loc_temp = seq_loc.copy()

for key,value in seq_name_format.items():

    seq_loc = [j+seq_loc_temp[-1] for j in range(1,value+1)]
    image = seq_loc.copy()
    categories = seq_loc.copy()
    is_seq_empty = 1
    empty_frame_loc = list()
    
    for j in range(1,value+1):
               
        loc = j+seq_loc_temp[-1]
        
        seq_frame_num = train_meta['images'][loc]['seq_frame_num']
        category = cate_name[file_name2.index(file_name[loc])]
        if category == 0:
            empty_frame_loc.append(seq_frame_num)
        else:
            is_seq_empty = 0
            categories[seq_frame_num] = category
            image[seq_frame_num] = train_meta['images'][loc]
        
    if is_seq_empty == 1:
        seq_loc_empty.append(i)
    else:
        categories = deletor(categories, empty_frame_loc)
        image = deletor(image, empty_frame_loc)
       
    # seq_loc = [i+seq_loc_temp[-1] for i in range(1,value+1)]
    # image = [train_meta['images'][i+seq_loc_temp[-1]] for i in range(1,value+1)]
    # category = [cate_name[file_name2.index(file_name[i+seq_loc_temp[-1]])] for i in range(1,value+1)]

    seq_loc_temp = seq_loc.copy()
    seq_all[i] = {'seq_id':key,'image':image,'category_id':categories, 'is_irr':False}
    i += 1
    if i%1000 ==0:
        print(i)

seq_all = deletor(seq_all, seq_loc_empty)

Dealing with sequences. I found some totally empty, others partly empty, but the majority is intact. I reckon empty pictures valuless, therefore, they have been removed from the re-organised variable: seq_all 

In [ ]:
categories = extractor(seq_all, 'category_id')
irr_cate_loc = list()

for i in range(len(categories)):
    category = categories[i]
    x_s = category[0]
    is_irr = 0
    for x in category:
        if x != x_s:
            is_irr = 1
            seq_all[i]['is_irr'] = True
            break
    if is_irr == 1:
        irr_cate_loc.append(i)

Another stuff intrigue: Some sequences contains more than one species， which means it cannot be described as only one category. So there is another tag in seq_all: 'is_irr', that is, is it irrgular. Luckily, most sequences are regular

In [ ]:
#with open('../metadata/metadata/seq_all.json', 'w') as f:
#   json.dump(seq_all, f)

seq_all saved for further analysis, and I deem the data structure of seq_all.json superior as iwildcam2022_train_annotations.json